In [1]:
import numpy as np
import tensorflow as tf

/Users/zhanglei115/dev/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Example dummy data from Rendle 2010 
# http://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf
# Stolen from https://github.com/coreylynch/pyFM
# Categorical variables (Users, Movies, Last Rated) have been one-hot-encoded 

x_data = np.matrix([
#    Users  |     Movies     |    Movie Ratings   | Time | Last Movies Rated
#   A  B  C | TI  NH  SW  ST | TI   NH   SW   ST  |      | TI  NH  SW  ST
    [1, 0, 0,  1,  0,  0,  0,   0.3, 0.3, 0.3, 0,     13,   0,  0,  0,  0 ],
    [1, 0, 0,  0,  1,  0,  0,   0.3, 0.3, 0.3, 0,     14,   1,  0,  0,  0 ],
    [1, 0, 0,  0,  0,  1,  0,   0.3, 0.3, 0.3, 0,     16,   0,  1,  0,  0 ],
    [0, 1, 0,  0,  0,  1,  0,   0,   0,   0.5, 0.5,   5,    0,  0,  0,  0 ],
    [0, 1, 0,  0,  0,  0,  1,   0,   0,   0.5, 0.5,   8,    0,  0,  1,  0 ],
    [0, 0, 1,  1,  0,  0,  0,   0.5, 0,   0.5, 0,     9,    0,  0,  0,  0 ],
    [0, 0, 1,  0,  0,  1,  0,   0.5, 0,   0.5, 0,     12,   1,  0,  0,  0 ]
])
# ratings
y_data = np.array([5, 3, 1, 4, 5, 1, 5])

# Let's add an axis to make tensoflow happy.
y_data.shape += (1, )

In [3]:
print(x_data.shape)
print(y_data.shape)

(7, 16)
(7, 1)


In [4]:
n, p = x_data.shape

# number of latent factors
k = 5

X = tf.placeholder('float', shape=[n, p])  # design matrix
y = tf.placeholder('float', shape=[n, 1])  # target vector

# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# interaction factors, randomly initialized 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimate of y, initialized to 0.
y_pred = tf.Variable(tf.zeros([n, 1]))

print('X_shape: {}'.format(X.shape))
print('V_shape: {}'.format(V.shape))

X_shape: (7, 16)
V_shape: (5, 16)


In [5]:
linear_terms = tf.add(w0, tf.reduce_sum(tf.multiply(W, X), 1, keepdims=True))

In [6]:
interactions = (tf.multiply(0.5,
                    tf.reduce_sum(tf.subtract(tf.pow(tf.matmul(X, tf.transpose(V)), 2),
                        tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))), 1, keep_dims=True)))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
y_pred = tf.add(linear_terms, interactions)

In [8]:
# L2 regularized sum of squares loss function over W and V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_pred)))
loss = tf.add(error, l2_norm)

In [9]:
eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

In [10]:
# that's a lot of iterations
N_EPOCHS = 1000
# Launch the graph.
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(N_EPOCHS):
        indices = np.arange(n)
        np.random.shuffle(indices)
        x_data, y_data = x_data[indices], y_data[indices]
        sess.run(optimizer, feed_dict={X: x_data, y: y_data})

    print('MSE: ', sess.run(error, feed_dict={X: x_data, y: y_data}))
    print('Loss (regularized error):', sess.run(loss, feed_dict={X: x_data, y: y_data}))
    print('Predictions:', sess.run(y_pred, feed_dict={X: x_data, y: y_data}))
    print('Learnt weights:', sess.run(W, feed_dict={X: x_data, y: y_data}))
    print('Learnt factors:', sess.run(V, feed_dict={X: x_data, y: y_data}))

MSE:  2.8889091e-05
Loss (regularized error): 0.0032499814
Predictions: [[4.99564  ]
 [4.993176 ]
 [4.99407  ]
 [0.9936371]
 [3.996399 ]
 [2.9935665]
 [0.9974238]]
Learnt weights: [ 0.1243737   0.19964918 -0.03787979  0.01391326 -0.03862222  0.22370568
  0.10294139  0.03036066  0.06911231  0.1133675   0.15240586  0.13388005
  0.1917422  -0.17157163  0.10294139  0.        ]
Learnt factors: [[ 0.13164376  0.24746239 -0.11788798  0.15289064 -0.24639721  0.23914546
   0.07790542 -0.00194411  0.05762508  0.06863816  0.22859074  0.27847663
   0.270541   -0.41267282  0.08678205  0.00170862]
 [-0.01340673 -0.00963179  0.00146053  0.00218098 -0.0187941   0.00295163
  -0.01480574 -0.01203586  0.00269673 -0.00384837 -0.00371918  0.00518827
   0.02520728 -0.0007851  -0.01370601 -0.01108399]
 [ 0.09146155  0.22232804 -0.17726585  0.12757531 -0.27285418  0.21869859
   0.07112934 -0.02408755  0.0531369   0.04090684  0.21313836  0.30551946
   0.23766786 -0.43542203  0.07199112 -0.00582941]
 [-0.097240